In [24]:
import configparser
from datetime import datetime
import time
import os
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, col, to_date, isnan, when, count
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql import types as t
from pyspark.sql.types     import IntegerType, TimestampType, DoubleType, StructType, StructField
from pyspark.sql.types     import *
from pyspark.sql.functions import to_timestamp
import boto3
from botocore.client import Config
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import json
import csv
from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

!pip install jupyters3
import jupyters3

"""^^^Install all tools and import all required libraries^^^"""

'^^^Install all tools and import all required libraries^^^'

In [43]:
config = configparser.ConfigParser()
config.read_file(open('dl.cfg'))

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

"""^^^Configure AWS access^^^"""

'^^^Configure AWS access^^^'

In [44]:
spark = SparkSession.builder\
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0")\
    .enableHiveSupport().getOrCreate()

print('Creating Spark session: COMPLETE')

"""^^^Creating Spark session for data processing, if it does not currently exist^^^"""

Creating Spark session: COMPLETE


'^^^Creating Spark session for data processing, if it does not currently exist^^^'

In [45]:
s3 = boto3.resource('s3')
for bucket in s3.buckets.all():
    print(bucket.name)

"""^^^ Utilize boto3 library to verify connection by printing out S3 buckets^^^"""

aws-emr-resources-795912878947-us-west-2
aws-glue-scripts-795912878947-us-west-2
aws-glue-temporary-795912878947-us-west-2
aws-logs-795912878947-us-west-2
capston-bucket2
dend-capstone-bucket2
dend-capstone-crypto-bucket
dend-capstone-output
dend-s3-buckey
im-cap
p4-buckey
p5-bucket


'^^^ Utilize boto3 library to verify connection by printing out S3 buckets^^^'

In [46]:
LTC_csv=spark.read.csv('s3a://dend-capstone-crypto-bucket/ltcusd.csv', header=True)
ETH_json=spark.read.json('s3a://capston-bucket2/ethusd.json')
LTC_csv.printSchema()
ETH_json.printSchema()
"""^^^Read in S3 data using Spark and print schema^^^"""

root
 |-- time: string (nullable = true)
 |-- open: string (nullable = true)
 |-- close: string (nullable = true)
 |-- high: string (nullable = true)
 |-- low: string (nullable = true)
 |-- volume: string (nullable = true)

root
 |-- results: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- close: double (nullable = true)
 |    |    |-- high: double (nullable = true)
 |    |    |-- low: double (nullable = true)
 |    |    |-- open: double (nullable = true)
 |    |    |-- time: long (nullable = true)
 |    |    |-- volume: double (nullable = true)



'^^^Read in S3 data using Spark and print schema^^^'

In [47]:
print(f'LTC_csv dataframe shape:',(LTC_csv.count(), len(LTC_csv.columns)))
LTC_csv.select([count(when(isnan(c), c)).alias(c) for c in LTC_csv.columns]).show()
"""^^^Data Quality check #1--Check shape of dataframe and if any columns contain NaN values^^^"""

LTC_csv dataframe shape: (1663435, 6)
+----+----+-----+----+---+------+
|time|open|close|high|low|volume|
+----+----+-----+----+---+------+
|   0|   0|    0|   0|  0|     0|
+----+----+-----+----+---+------+



'^^^Data Quality check #1--Check shape of dataframe and if any columns contain NaN values^^^'

In [31]:
def process_ltc(spark, input_data, output_data):
    print('Processing xrp data from S3 bucket...')
    """
    This function will:  Extract and process data to 
    create (df)
    """
    
    
    ltc_data = input_data
    df = LTC_csv
    print('Reading ltc data from S3 bucket: COMPLETE')
    
    """^^^Read in data from S3 bucket and assign it as df^^^"""


In [32]:
future_pred=30 #References how far out the prediction model will predict
#coin_to_predict= "ltcusd"    

main_df=pd.DataFrame()
df=LTC_csv.toPandas()
df.set_index("time", inplace=True)
    
if len(main_df)== 0:
    main_df=df
else:
    main_df = main_df.join(df)
        
"""^^^Convert Spark dataframe to pandas dataframe & set index as 'time'^^^"""


"^^^Convert Spark dataframe to pandas dataframe & set index as 'time'^^^"

In [33]:
main_df.fillna(method='bfill', inplace=True)
main_df.dropna(inplace=True)
"""^^^Back fill close data where values were NaN^^^"""



main_df['LTC_future']= main_df["close"].shift(-future_pred)
main_df.dropna(inplace=True)
ltc_df=main_df[['close', 'LTC_future']]
ltc_df.head()
"""^^^Create new df as ltc_df with only 'close' and 'LTC_future' columns extracted from main_df^^^"""

"^^^Create new df as ltc_df with only 'close' and 'LTC_future' columns extracted from main_df^^^"

In [34]:
"""___Begin building model for LTC future prediction___"""

X= np.array(ltc_df.drop(['LTC_future'], 1))[:-future_pred]
print('x:', X)
Y=np.array(ltc_df['LTC_future'])[:-future_pred]
print('y:',Y)

#Split data 75% training
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)
"""^^^Split training and test data with 75/25 split^^^"""

# Decision tree regressor model
tree= DecisionTreeRegressor().fit(x_train, y_train)
#Linear Regression model
lr= LinearRegression().fit(x_train, y_train)
"""^^^Create Decision Tree and Linear Regresion Models^^^"""

#Get last rows of future
x_future= ltc_df.drop(['LTC_future'], 1)[:-future_pred]
x_future= x_future.tail(future_pred)
x_future= np.array(x_future)
x_future

x: [['3.1491']
 ['3.1491']
 ['3.1491']
 ..., 
 ['57.695']
 ['57.706']
 ['57.716']]
y: ['3.05' '3.05' '3.05' ..., '58.03100000000001' '58.095' '58.066']


array([['57.737'],
       ['57.7'],
       ['57.701'],
       ['57.745'],
       ['57.746'],
       ['57.799'],
       ['57.775'],
       ['57.784'],
       ['57.883'],
       ['57.878'],
       ['57.841'],
       ['57.867'],
       ['57.95'],
       ['57.95'],
       ['57.93'],
       ['57.84'],
       ['57.8'],
       ['57.8'],
       ['57.808'],
       ['57.86'],
       ['57.8'],
       ['57.8'],
       ['57.808'],
       ['57.782'],
       ['57.839'],
       ['57.833'],
       ['57.73'],
       ['57.695'],
       ['57.706'],
       ['57.716']], dtype=object)

In [35]:
#Show model tree prediction
tree_pred= tree.predict(x_future)
print(f'Decission Tree: {tree_pred}')
print()
#Show Linear model pred
lr_pred= lr.predict(x_future)
print(f'Linear Regression: {lr_pred}')


predictions= tree_pred
valid_tree= ltc_df[X.shape[0]:]
valid_tree['Predictions']= predictions
valid_tree= valid_tree.apply(pd.to_numeric)
valid_tree['Error_%']=(valid_tree['LTC_future']-valid_tree['Predictions'])/valid_tree['LTC_future']*(100)
valid_tree['LTC_future'].dtype

"""^^^Create new column 'Error_%' to view the percentage of error between the predictions and actual future value.
[((future value) - (prediction value) / (future value))* 100] ^^^"""

valid_tree.reset_index(inplace=True) 
valid_tree.head()
"""^^^Reset index for valid_tree dataframe^^^"""

predictions= lr_pred
valid_lr= ltc_df[X.shape[0]:]
valid_lr['Predictions']= predictions
valid_lr= valid_lr.apply(pd.to_numeric)
valid_lr['Error_%']=(valid_lr['LTC_future']-valid_lr['Predictions'])/valid_lr['LTC_future']*(100)
valid_lr.reset_index(inplace=True)
valid_lr.head()
"""^^^Create new column 'Error_%' to view the percentage of error between the predictions and actual future value.
[((future value) - (prediction value) / (future value))* 100] ^^^"""
"""^^^Reset index for valid_tree dataframe^^^"""


valid_tree['time'] = pd.to_datetime(valid_tree['time'],unit='ms')
valid_lr['time'] = pd.to_datetime(valid_lr['time'],unit='ms')
"""^^^Convert the 'time' column from epoch seconds to pandas datetime^^^"""

Decission Tree: [ 57.85740909  57.80831304  57.82542857  57.69136364  57.77226316
  57.83123077  57.8165      57.74213333  57.94093333  57.87541379
  58.08811765  57.97028571  57.97265079  57.97265079  57.86448718
  57.88075676  57.86157265  57.86157265  57.87467742  57.90087234
  57.86157265  57.86157265  57.87467742  57.81052632  57.784       57.67472
  57.70281667  57.6299      57.6514      57.56809091]

Linear Regression: [ 57.74037452  57.70338218  57.70438197  57.74837287  57.74937266
  57.8023617   57.77836666  57.7873648   57.88634432  57.88134536
  57.84435301  57.87034763  57.95333046  57.95333046  57.9333346
  57.84335322  57.80336149  57.80336149  57.81135984  57.86334908
  57.80336149  57.80336149  57.81135984  57.78536522  57.84235342
  57.83635467  57.73337597  57.69838321  57.70938094  57.71937887]


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


"^^^Convert the 'time' column from epoch seconds to pandas datetime^^^"

In [36]:
valid_tree_spark = spark.createDataFrame(valid_tree)
valid_tree_spark=valid_tree_spark.withColumn('time', col('time').cast('Timestamp'))
valid_tree_spark=valid_tree_spark.withColumn('close', col('close').cast('Float'))
valid_tree_spark=valid_tree_spark.withColumn('LTC_future', col('LTC_future').cast('Float'))
valid_tree_spark=valid_tree_spark.withColumn('Predictions', col('Predictions').cast('Float'))
valid_tree_spark=valid_tree_spark.withColumn('Error_%', col('Error_%').cast('Float'))
valid_tree_spark.printSchema()
valid_tree_spark.show()
"""^^^Convert data back to Spark dataframes and cast the appropriate data types and print schema to verify^^^"""



valid_lr_spark = spark.createDataFrame(valid_lr)
valid_lr_spark=valid_lr_spark.withColumn('time', col('time').cast('Timestamp'))
valid_lr_spark=valid_lr_spark.withColumn('close', col('close').cast('Float'))
valid_lr_spark=valid_lr_spark.withColumn('LTC_future', col('LTC_future').cast('Float'))
valid_lr_spark=valid_lr_spark.withColumn('Predictions', col('Predictions').cast('Float'))
valid_lr_spark=valid_lr_spark.withColumn('Error_%', col('Error_%').cast('Float'))
valid_lr_spark.printSchema()
valid_lr_spark.show()
"""^^^Convert data back to Spark dataframes and cast the appropriate data types and print schema to verify^^^"""

root
 |-- time: timestamp (nullable = true)
 |-- close: float (nullable = true)
 |-- LTC_future: float (nullable = true)
 |-- Predictions: float (nullable = true)
 |-- Error_%: float (nullable = true)

+-------------------+------+----------+-----------+-----------+
|               time| close|LTC_future|Predictions|    Error_%|
+-------------------+------+----------+-----------+-----------+
|2020-08-27 07:26:00|57.869|     58.08|   57.85741|  0.3832488|
|2020-08-27 07:29:00|58.043|    58.083|   57.80831| 0.47292143|
|2020-08-27 07:36:00|58.135|    58.052|  57.825428| 0.39029047|
|2020-08-27 07:37:00|58.205|    58.124|  57.691364| 0.74433345|
|2020-08-27 07:38:00| 58.41|     58.12|  57.772263|  0.5983084|
|2020-08-27 07:39:00|58.449|    58.101|   57.83123| 0.46431082|
|2020-08-27 07:40:00|58.222|    58.082|    57.8165| 0.45711237|
|2020-08-27 07:42:00| 58.38|    58.088|  57.742134| 0.59541845|
|2020-08-27 07:43:00| 58.37|    58.088|  57.940933| 0.25317907|
|2020-08-27 07:46:00|58.286|  

'^^^Convert data back to Spark dataframes and cast the appropriate data types and print schema to verify^^^'

In [39]:
print(f'valid_tree_spark dataframe shape:',(valid_tree_spark.count(), len(valid_tree_spark.columns)))
#valid_tree_spark.select([count(when(isnan(c), c)).alias(c) for c in valid_tree_spark.columns]).show()

print(f'valid_lr_spark dataframe shape:',(valid_lr_spark.count(), len(valid_lr_spark.columns)))
#valid_lr_spark.select([count(when(isnan(c), c)).alias(c) for c in valid_lr_spark.columns]).show()

"""^^^Data Quality check #2-- Checking shape of dataframes^^^"""

valid_tree_spark dataframe shape: (30, 5)
valid_lr_spark dataframe shape: (30, 5)


'^^^Data Quality check #2-- Checking shape of dataframes^^^'

In [40]:
valid_tree_spark.write.partitionBy('time').parquet("s3a://dend-capstone-output/"+ 'valid_tree_spark')
valid_lr_spark.write.partitionBy('time').parquet("s3a://dend-capstone-output/"+ 'valid_lr_spark')
print('Write parquet files to S3 files & partition by time: COMPLETE')

"""^^^Write decision tree and linear regression tables to S3 bucket as parquet files partitioned on 'time'^^^"""

AnalysisException: 'path s3a://dend-capstone-output/valid_tree_spark already exists.;'

In [41]:
def main():
    spark
    input_data = "s3a://dend-capstone-crypto-bucket/"
    output_data = "s3a://dend-capstone-output/"
    process_ltc
    #write_parq()
    
    #process_xrp(spark, input_data, output_data)
    print('ETL PROCESSING COMPLETE!')
    
    
    
if __name__=="__main__":
    main()

ETL PROCESSING COMPLETE!
